In [2]:

import pandas as pd
import json

# Read the JSON file and convert it to a DataFrame
with open('./totalProposalsMadeByUser.json') as f:
    json_data = pd.DataFrame.from_dict(json.load(f), orient='index', columns=['Totalproposalmadebyuser'])
    json_data.index.name = 'Address'

# Read the existing DataFrame 'df' from the CSV file
df = pd.read_csv('./export (11).csv')

# Merge the two DataFrames on the 'Address' column
merged_df = df.merge(json_data, on='Address', how='left')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_df.csv', index=False)


In [4]:

merged_df.fillna(0, inplace=True)

In [5]:

merged_df.head()

,Address,Name,Is delegator,Avatar,Space,Ens name,Follow Count,Follows Space,Total Votes,Proposal Interaction,Proposal Interaction In Space,Proposal Interaction outside of Space,Total NFT`s,Total Rewards NFT`s,User is delegator,Totalproposalmadebyuser
0,0x329c54289Ff5D6B7b7daE13592C6B1EDA1543eD4,0,0,0,aave.eth,aavechan.eth,7,1,3.392825e+07,527,256,271,1,0,0,50.0
1,0xB83b3e9C8E3393889Afb272D354A7a3Bd1Fbcf5C,0,0,0,aave.eth,lbsblockchain.eth,1,1,1.485489e+07,184,184,0,0,0,0,2.0
2,0x13BDaE8c5F0fC40231F0E6A4ad70196F59138548,0,0,0,aave.eth,michiganblockchain.eth,2,1,7.785688e+06,151,96,55,0,0,0,1.0
3,0xB933AEe47C438f22DE0747D57fc239FE37878Dd1,0,0,0,aave.eth,wintermutegovernance.eth,17,1,6.334191e+06,307,151,156,23,0,0,0.0
4,0x55B16934C3661E1990939bC57322554d9B09f262,Llama Proposal Power,0,ipfs://bafkreidlvzco3qpbpwir5xmln4zygbyxtcfixn...,aave.eth,0,2,1,3.722838e+06,57,57,0,0,0,0,0.0


In [6]:

df = merged_df.copy()

In [17]:
import pandas as pd
import math

# Assuming you have your dataset loaded into a pandas DataFrame called 'df'

# Selecting the relevant columns including 'Totalproposalmadebyuser'
selected_columns = ['Proposal Interaction In Space', 'Total NFT`s', 'Total Votes', 'Totalproposalmadebyuser']
df_selected = df[selected_columns].copy()

# Convert selected columns to integers
df_selected[selected_columns] = df_selected[selected_columns].astype(int)

# Dictionary to store the intervals for each column
intervals_dict = {}

# Suggest intervals for each column
for col in selected_columns:
    # Create intervals for 'Totalproposalmadebyuser' column
    if col == 'Totalproposalmadebyuser':
        num_intervals = 5
        min_val = df_selected['Totalproposalmadebyuser'].min()
        max_val = df_selected['Totalproposalmadebyuser'].max()
        interval_size = math.ceil((max_val - min_val + 1) / num_intervals)
        intervals = [(min_val + i * interval_size, min(min_val + (i + 1) * interval_size, max_val)) for i in range(num_intervals)]
        
    else:
        # For other columns, use 5 intervals with equal interval size
        num_intervals = 5
        min_val = df_selected[col].min()
        max_val = df_selected[col].max()
        interval_size = math.ceil((max_val - min_val + 1) / num_intervals)
        intervals = [(min_val + i * interval_size, min(min_val + (i + 1) * interval_size, max_val)) for i in range(num_intervals)]
    
    # Add the additional interval for values greater than the maximum value
    intervals.append((">" + str(max_val),))
    
    intervals_dict[col] = intervals

# Create a DataFrame to store the intervals
interval_df = pd.DataFrame(intervals_dict)

# Function to calculate the score for each interval
def calculate_score(interval_index):
    total_intervals = len(interval_df) - 1
    score = interval_index * (100 / total_intervals)
    return int(round(score))

# Create a DataFrame to store the scores
score_df = pd.DataFrame(columns=selected_columns)

# Calculate and assign scores for each interval
for col in selected_columns:
    scores = [calculate_score(interval_df[col].tolist().index(interval)) for interval in interval_df[col]]
    score_df[col] = scores

# Printing the DataFrames with the suggested intervals and scores for each column
print("Interval DataFrame:")
print(interval_df)
print("\nScore DataFrame:")
print(score_df)


Interval DataFrame:
  Proposal Interaction In Space Total NFT`s           Total Votes  \
0                       (0, 66)     (0, 11)          (0, 6785651)   
1                     (66, 132)    (11, 22)   (6785651, 13571302)   
2                    (132, 198)    (22, 33)  (13571302, 20356953)   
3                    (198, 264)    (33, 44)  (20356953, 27142604)   
4                    (264, 328)    (44, 50)  (27142604, 33928250)   
5                       (>328,)      (>50,)          (>33928250,)   

  Totalproposalmadebyuser  
0                 (0, 12)  
1                (12, 24)  
2                (24, 36)  
3                (36, 48)  
4                (48, 55)  
5                  (>55,)  

Score DataFrame:
   Proposal Interaction In Space  Total NFT`s  Total Votes  \
0                              0            0            0   
1                             20           20           20   
2                             40           40           40   
3                             60  